In [3]:
print('hello')

hello


In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("Covid_Hive_HDFS") \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()

In [5]:
# Для проверки создадим DataFrame и выведем данные
data = [("Alice", 34), ("Bob", 45), ("Cathy", 29)]
df = spark.createDataFrame(data, ["Name", "Age"])
df.show()


+-----+---+
| Name|Age|
+-----+---+
|Alice| 34|
|  Bob| 45|
|Cathy| 29|
+-----+---+



In [24]:
csv_path = "hdfs://hadoop-namenode:9000/user/hadoop/metadata/metadata_cleaned.csv"

df = spark.read.option("header", True).option("inferSchema", True).csv(csv_path)

In [25]:
df.show(5)

+---------+------+---+----+--------+----+--------+----------------+--------------------+------+--------------------+--------------------+-------+--------------------+-----------+---------+
|patientid|offset|sex| age| finding|view|modality|            date|            location|folder|            filename|                 url|license|      clinical_notes|  age_group|has_covid|
+---------+------+---+----+--------+----+--------+----------------+--------------------+------+--------------------+--------------------+-------+--------------------+-----------+---------+
|        2|   0.0|  M|65.0|COVID-19|  PA|   X-ray|January 22, 2020|Cho Ray Hospital,...|images|auntminnie-a-2020...|https://www.nejm....|   None|On January 22, 20...|     Senior|     True|
|        2|   3.0|  M|65.0|COVID-19|  PA|   X-ray|January 25, 2020|Cho Ray Hospital,...|images|auntminnie-b-2020...|https://www.nejm....|   None|On January 22, 20...|     Senior|     True|
|        2|   5.0|  M|65.0|COVID-19|  PA|   X-ray|Janua

In [28]:
spark.sql("CREATE DATABASE IF NOT EXISTS covid_db")


25/07/07 18:43:09 WARN ObjectStore: Failed to get database covid_db, returning NoSuchObjectException
25/07/07 18:43:09 WARN ObjectStore: Failed to get database covid_db, returning NoSuchObjectException
25/07/07 18:43:09 WARN ObjectStore: Failed to get database covid_db, returning NoSuchObjectException


DataFrame[]

In [31]:
output_path = "hdfs://hadoop-namenode:9000/user/hadoop/metadata_parquet/"

df.write \
  .mode("overwrite") \
  .parquet(output_path)


In [32]:
parquet_path = "hdfs://hadoop-namenode:9000/user/hadoop/metadata_parquet/"

df_parquet = spark.read.parquet(parquet_path)
df_parquet.show(5)


+---------+------+---+----+--------+----+--------+----------------+--------------------+------+--------------------+--------------------+-------+--------------------+-----------+---------+
|patientid|offset|sex| age| finding|view|modality|            date|            location|folder|            filename|                 url|license|      clinical_notes|  age_group|has_covid|
+---------+------+---+----+--------+----+--------+----------------+--------------------+------+--------------------+--------------------+-------+--------------------+-----------+---------+
|        2|   0.0|  M|65.0|COVID-19|  PA|   X-ray|January 22, 2020|Cho Ray Hospital,...|images|auntminnie-a-2020...|https://www.nejm....|   None|On January 22, 20...|     Senior|     True|
|        2|   3.0|  M|65.0|COVID-19|  PA|   X-ray|January 25, 2020|Cho Ray Hospital,...|images|auntminnie-b-2020...|https://www.nejm....|   None|On January 22, 20...|     Senior|     True|
|        2|   5.0|  M|65.0|COVID-19|  PA|   X-ray|Janua

In [36]:
spark.sql("DROP TABLE IF EXISTS metadata")

DataFrame[]

In [38]:
spark.sql("""
CREATE TABLE IF NOT EXISTS metadata_bucketed (
  patientid STRING,
  offset INT,
  sex STRING,
  age INT,
  finding STRING,
  view STRING,
  modality STRING,
  location STRING,
  folder STRING,
  filename STRING,
  url STRING,
  license STRING,
  clinical_notes STRING
)
PARTITIONED BY (date STRING)
CLUSTERED BY (patientid) INTO 8 BUCKETS
STORED AS ORC
TBLPROPERTIES ("transactional"="true")
""")


25/07/07 18:58:35 WARN HiveMetaStore: Location: file:/workspace/spark-warehouse/metadata_bucketed specified for non-external table:metadata_bucketed


DataFrame[]

In [39]:
df_parquet.createOrReplaceTempView("temp_metadata")

spark.sql("SET hive.exec.dynamic.partition = true")
spark.sql("SET hive.exec.dynamic.partition.mode = nonstrict")

spark.sql("""
INSERT OVERWRITE TABLE metadata_bucketed PARTITION (date)
SELECT
  patientid,
  offset,
  sex,
  age,
  finding,
  view,
  modality,
  location,
  folder,
  filename,
  url,
  license,
  clinical_notes,
  date
FROM temp_metadata
""")


25/07/07 18:59:08 WARN SetCommand: 'SET hive.exec.dynamic.partition=true' might not work, since Spark doesn't support changing the Hive config dynamically. Please pass the Hive-specific config by adding the prefix spark.hadoop (e.g. spark.hadoop.hive.exec.dynamic.partition) when starting a Spark application. For details, see the link: https://spark.apache.org/docs/latest/configuration.html#dynamically-loading-spark-properties.
25/07/07 18:59:08 WARN SetCommand: 'SET hive.exec.dynamic.partition.mode=nonstrict' might not work, since Spark doesn't support changing the Hive config dynamically. Please pass the Hive-specific config by adding the prefix spark.hadoop (e.g. spark.hadoop.hive.exec.dynamic.partition.mode) when starting a Spark application. For details, see the link: https://spark.apache.org/docs/latest/configuration.html#dynamically-loading-spark-properties.
25/07/07 18:59:10 WARN TaskSetManager: Lost task 0.0 in stage 10.0 (TID 22) (172.18.0.5 executor 0): org.apache.spark.Spark

Py4JJavaError: An error occurred while calling o80.sql.
: org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to file:/workspace/spark-warehouse/metadata_bucketed/.spark-staging-07386fa2-1467-4988-ad49-638f7784cf81. SQLSTATE: 58030
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:772)
	at org.apache.spark.sql.execution.datasources.FileFormatDataWriter.enrichWriteError(FileFormatDataWriter.scala:92)
	at org.apache.spark.sql.execution.datasources.FileFormatDataWriter.writeWithMetrics(FileFormatDataWriter.scala:103)
	at org.apache.spark.sql.execution.datasources.FileFormatDataWriter.writeWithIterator(FileFormatDataWriter.scala:111)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.$anonfun$executeTask$1(FileFormatWriter.scala:406)
	at org.apache.spark.util.Utils$.tryWithSafeFinallyAndFailureCallbacks(Utils.scala:1323)
	at org.apache.spark.sql.execution.datasources.FileFormatWrite